In [22]:
# Goal is to reduce the average wait time to 10 minutes or less

# STEPS
# arrive at the theater
# get in line to buy a ticket
# buy a ticket
# wait in line to have their ticket checked
# have their tickets checked
# decide whether or not to get in line for concessions
# buy concessions or go directly to seat

import simpy
import random
import statistics


In [43]:
wait_times = []# create an empty list to hold the wait times we are intereseted in

# create the environment class, the theater as a whole = self
class Theater(object): 
    def __init__(self,env, num_cashiers, num_servers, num_ushers): # Create the constructor and pass in the simpy environment
        self.env = env # set it as an environemnt
        # set the paramenters
        self.cashier = simpy.Resource(env, num_cashiers) # simpy resource that will contain the environment and number of cashiers
        self.server = simpy.Resource(env, num_servers)
        self.usher = simpy.Resource(env, num_ushers)
        
   # add another method in the class
    def purchase_ticket(self, moviegoer): #request a cashier
        # estimate using previous data
        yield self.env.timeout(random.randint(1,3))
        
    def check_ticket(self,moviegoer): #request an usher
        yield self.env.timeout(3/60)
        
    def sell_food(self, moviegoer): #request a server
        yield self.env.timeout(random.randint(1,5))
        
        

In [44]:
# moving through the environment
# many moviegoers can use the same cashier but a cashier can only help one moviegoer at a time
def go_to_movies (env, moviegoer, theater): # env - control the moviegoer, moviegoer - each person as they move through the environment, theater - gives you access to processes you defined in the overall class
    arrival_time = env.now # moviegoer arrives at the theater
    
    # the with statement automatically releases the moviegoer once the process is complete and the cashier is not available to take on a new customer
    with theater.cashier.request() as request: #request to use the cashier
        yield request # moviegoer waits for the cashier to be available 
        yield env.process(theater.purchase_ticket(moviegoer)) # moviegoer uses available cashier to complete the process (purchasing a ticket)
        
    with theater.usher.request() as request:
        yield request
        yield env.process(theater.check_ticket(moviegoer))
            
            # Use boolean values to randomly decide whether a moviegoer will buy food or not
    if random.choice([True, False]): #True - moviegoe will request a server and order
        with theater.server.request() as request:
            yield request
            yield env.process(theater.sell_food(moviegoer))
    # the moviegoer heads to the theater
    wait_times.append (env.now - arrival_time)

In [45]:
#create an instance of the the theater
def run_theater(env, num_cashiers, num_servers, num_ushers):
    theater = Theater(env, num_cashiers, num_servers, num_ushers)
    
    for moviegoer in range(3):
        env.process(go_to_movies(env, moviegoer, theater))

    while True:
        yield env.timeout(0.20)
        moviegoer += 1
        env.process(go_to_movies(env, moviegoer, theater))
        

In [50]:
def get_average_wait_time(wait_times):
    average_wait = statistics.mean(wait_times)
    minutes, frac_minutes = divmod(average_wait, 1)
    seconds = frac_minutes * 60
    return round(minutes), round(seconds)

In [51]:
def get_user_input():
    num_cashiers = input("Input # of cashiers working: ")
    num_servers = input("Input # of servers working: ")
    num_ushers = input("Input # of ushers working: ")
    
    params = [num_cashiers, num_servers, num_ushers]
    if all(str(i).isdigit() for i in params):
        params = [int(x) for x in params]
    else:
        print("Could not parse input. The simulation will use default values: ",
             "\n1 cashier, 1 server, 1 usher.",)
        params = [1,1,1]
    return params

In [54]:
def main():
    random.seed(42)
    num_cashiers, num_servers, num_ushers = get_user_input()
  
    
    env = simpy.Environment()
    env.process(run_theater(env, num_cashiers, num_servers, num_ushers))
    env.run(until = 90)
    
    mins, secs = get_average_wait_time(wait_times)
    print("Running simulation ...", f"\n The average wait time is {mins} minutes and {secs} seconds.",)

In [56]:
if __name__ == '__main__':
    main()

Input # of cashiers working: 3
Input # of servers working: 4
Input # of ushers working: 4
Running simulation ... 
 The average wait time is 37 minutes and 59 seconds.
